# Installation and dependencies

In [6]:
%pip install tensorflow==2.14.1 opencv-python matplotlib

Note: you may need to restart the kernel to use updated packages.


In [7]:
# Standard dependencies
import cv2
import os
import random
import numpy as np
import matplotlib.pyplot as plt

In [15]:
# Tensorflow dependencies
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

# Creating data directories

In [19]:
# Setting up dirs
POS_DIR = os.path.join("data", "positive")
NEG_DIR = os.path.join("data", "negative")
ANC_DIR = os.path.join("data", "anchor")

# Creating dirs
os.makedirs(POS_DIR)
os.makedirs(NEG_DIR)
os.makedirs(ANC_DIR)